In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
import pandas as pd

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client(auth_type="sso")

In [3]:
workspace_name="helper-demo-workspace-1"
pipeline_name="helper-demo-pipeline-1"
model_name="helper-demo-model-1"
model_file_name="./models/rf_model.onnx"

## get_workspace

### Error when workspace does not exist

In [4]:
workspace = wl.get_workspace(name=workspace_name)

Exception: Error: Workspace with name helper-demo-workspace-1 does not exist. If you would like to create one, send in the request with `create_if_not_exist` flag set to True.

### Create new workspace with `create_if_not_exist` flag

In [5]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
workspace

{'name': 'helper-demo-workspace-1', 'id': 18, 'archived': False, 'created_by': '464316c9-45dd-4926-ba40-c3817485fb9b', 'created_at': '2024-04-04T18:03:33.453716+00:00', 'models': [], 'pipelines': []}

In [6]:
wl.set_current_workspace(workspace)

{'name': 'helper-demo-workspace-1', 'id': 18, 'archived': False, 'created_by': '464316c9-45dd-4926-ba40-c3817485fb9b', 'created_at': '2024-04-04T18:03:33.453716+00:00', 'models': [], 'pipelines': []}

### Return existing workspace

In [7]:
workspace = wl.get_workspace(name=workspace_name)
workspace

{'name': 'helper-demo-workspace-1', 'id': 18, 'archived': False, 'created_by': '464316c9-45dd-4926-ba40-c3817485fb9b', 'created_at': '2024-04-04T18:03:33.453716+00:00', 'models': [], 'pipelines': []}

## create model and pipeline

In [8]:
helper_model = (wl.upload_model(model_name, 
                                        model_file_name, 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

In [9]:
helper_model.version()

'522e5e85-74f5-44d3-a734-4dfe2f70d790'

## get_model

### Error when model not found

In [10]:
wl.get_model(name="no-such-model")

Exception: Error: A model with the name no-such-model does not exist.

In [11]:
wl.get_model(name=model_name)

Name,helper-demo-model-1
Version,522e5e85-74f5-44d3-a734-4dfe2f70d790
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-04-Apr 18:03:38


In [12]:
wl.get_model(name=model_name, version="no-such-version")

Exception: Error: A model with the version no-such-version not found in this workspace.

In [14]:
model_1 = wl.get_model(name=model_name, version="522e5e85-74f5-44d3-a734-4dfe2f70d790")
model_1

Name,helper-demo-model-1
Version,522e5e85-74f5-44d3-a734-4dfe2f70d790
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-04-Apr 18:03:38


## get_pipeline

In [15]:
pipeline = wl.build_pipeline(pipeline_name)

### Error when pipeline doesn't exist

In [16]:
wl.get_pipeline(name="no-such-pipeline")

Exception: Pipeline no-such-pipeline not found in this workspace.

In [17]:
wl.get_pipeline(name=pipeline_name)

name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:04:09.143113+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,1647a8bd-4ed8-45bc-a2b8-b14ace57e490
steps,
published,False


### Error when version doesn't exist

In [18]:
wl.get_pipeline(name=pipeline_name, version="no-such-version")

Exception: Pipeline version no-such-version not found in this workspace.

In [21]:
wl.get_pipeline(name=pipeline_name, version="1647a8bd-4ed8-45bc-a2b8-b14ace57e490")

name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:04:09.143113+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,1647a8bd-4ed8-45bc-a2b8-b14ace57e490
steps,
published,False


### use returned model and pipeline from get_model / get_pipeline

In [22]:
mainpipeline = wl.get_pipeline(name=pipeline_name, version="1647a8bd-4ed8-45bc-a2b8-b14ace57e490")
mainpipeline.clear()

model_1 = wl.get_model(name=model_name, version="522e5e85-74f5-44d3-a734-4dfe2f70d790")
mainpipeline.add_model_step(model_1)
mainpipeline

name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:04:09.143113+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,1647a8bd-4ed8-45bc-a2b8-b14ace57e490
steps,
published,False


In [23]:
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
mainpipeline.deploy(deployment_config = deploy_config)

Waiting for deployment - this will take up to 45s ............. ok


name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:04:54.874386+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"e2ff1174-05e8-4ea0-9a2d-e2759e6b1707, 1647a8bd-4ed8-45bc-a2b8-b14ace57e490"
steps,helper-demo-model-1
published,False


In [27]:
mainpipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.116.64.162',
   'name': 'engine-5798df5fd7-nnmcx',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'helper-demo-pipeline-1',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'helper-demo-model-1',
      'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6',
      'status': 'Running',
      'version': '522e5e85-74f5-44d3-a734-4dfe2f70d790'}]}}],
 'engine_lbs': [{'ip': '10.116.64.161',
   'name': 'engine-lb-7bcbb95695-695k2',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

In [28]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-04-04 18:05:35.870,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0,...",[718013.7],0


In [29]:
mainpipeline.versions()

In [30]:
mainpipeline.undeploy()

Waiting for undeployment - this will take up to 45s ..................................... ok


name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:04:54.874386+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"e2ff1174-05e8-4ea0-9a2d-e2759e6b1707, 1647a8bd-4ed8-45bc-a2b8-b14ace57e490"
steps,helper-demo-model-1
published,False


### Get existing pipeline version and redeploy that specific version

#### Creating new versions

In [31]:
new_model = (wl.upload_model("helper-model-replace", 
                                        "./models/gbr_model.onnx", 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )
new_model

Name,helper-model-replace
Version,fec3642f-e990-4f44-aaa8-8ada296206bd
File Name,gbr_model.onnx
SHA,ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-04-Apr 18:07:40


In [33]:
mainpipeline.clear()
mainpipeline.add_model_step(new_model)
mainpipeline.steps()

[{'ModelInference': {'models': [{'name': 'helper-model-replace', 'version': 'fec3642f-e990-4f44-aaa8-8ada296206bd', 'sha': 'ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a'}]}}]

In [34]:
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
mainpipeline.deploy(deployment_config = deploy_config)

Waiting for deployment - this will take up to 45s ........... ok


name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:09:08.209764+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"adbda107-649f-447d-9b4f-91684b37cc37, e2ff1174-05e8-4ea0-9a2d-e2759e6b1707, 1647a8bd-4ed8-45bc-a2b8-b14ace57e490"
steps,helper-model-replace
published,False


In [37]:
mainpipeline.undeploy()

 ok


name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:09:08.209764+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"adbda107-649f-447d-9b4f-91684b37cc37, e2ff1174-05e8-4ea0-9a2d-e2759e6b1707, 1647a8bd-4ed8-45bc-a2b8-b14ace57e490"
steps,helper-model-replace
published,False


## Get specific version and redeploy

In [39]:
wl.search_pipelines(pipeline_name)

name,version,creation_time,last_updated_time,deployed,tags,steps
helper-demo-pipeline-1,adbda107-649f-447d-9b4f-91684b37cc37,2024-04-Apr 18:09:08,2024-04-Apr 18:09:08,False,,helper-model-replace
helper-demo-pipeline-1,e2ff1174-05e8-4ea0-9a2d-e2759e6b1707,2024-04-Apr 18:04:54,2024-04-Apr 18:04:54,(unknown),,helper-demo-model-1
helper-demo-pipeline-1,1647a8bd-4ed8-45bc-a2b8-b14ace57e490,2024-04-Apr 18:04:09,2024-04-Apr 18:04:09,(unknown),,


In [40]:
pipeline_with_version_set = wl.get_pipeline(name=pipeline_name, version="e2ff1174-05e8-4ea0-9a2d-e2759e6b1707")

In [41]:
pipeline_with_version_set._pipeline_version_to_deploy

name,helper-demo-pipeline-1
version,e2ff1174-05e8-4ea0-9a2d-e2759e6b1707
creation_time,2024-04-Apr 18:04:54
last_updated_time,2024-04-Apr 18:04:54
deployed,False
tags,
steps,helper-demo-model-1


In [42]:
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
pipeline_with_version_set.deploy(deployment_config = deploy_config)

Waiting for deployment - this will take up to 45s .................. ok


name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:16:05.302849+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"94e9989c-4ce0-4d9e-9308-0fa77c6f4049, adbda107-649f-447d-9b4f-91684b37cc37, e2ff1174-05e8-4ea0-9a2d-e2759e6b1707, 1647a8bd-4ed8-45bc-a2b8-b14ace57e490"
steps,helper-demo-model-1
published,False


In [43]:
pipeline_with_version_set.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,helper-demo-pipeline-1
created,2024-04-04 18:04:09.143113+00:00
last_updated,2024-04-04 18:16:05.302849+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"94e9989c-4ce0-4d9e-9308-0fa77c6f4049, adbda107-649f-447d-9b4f-91684b37cc37, e2ff1174-05e8-4ea0-9a2d-e2759e6b1707, 1647a8bd-4ed8-45bc-a2b8-b14ace57e490"
steps,helper-demo-model-1
published,False
